In [1]:
import os, re, requests

from openai import OpenAI
from ddgs import DDGS
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
def get_weather(location: str):
    """
    Fetches the current weather for a location using the Open-Meteo API.
    It first converts the location name to coordinates, then gets the weather.
    """
    try:
        # Step 1: Geocoding (convert city name to lat/lon)
        geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={location}&count=1"
        geo_response = requests.get(geo_url)
        geo_response.raise_for_status()
        geo_data = geo_response.json()

        if not geo_data.get("results"):
            return f"Could not find coordinates for '{location}'."

        res = geo_data["results"][0]
        lat, lon = res["latitude"], res["longitude"]
        city_name = res["name"]

        # Step 2: Get weather using coordinates
        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code"
        weather_response = requests.get(weather_url)
        weather_response.raise_for_status()
        weather_data = weather_response.json()
        
        temp = weather_data['current']['temperature_2m']
        weather_code = weather_data['current']['weather_code']

        return f"The current temperature in {city_name} is {temp}°C (weather code: {weather_code})."

    except requests.exceptions.RequestException as e:
        return f"Error fetching data: {e}"
    except (KeyError, IndexError):
        return f"Could not process weather data for '{location}'."

In [5]:
def ddg_search(query: str):
    """
    Performs a search using the duckduckgo-search library to get a snippet.
    """
    try:
        # max_results=3 gives us a few options to find a good snippet
        results = DDGS().text(query, max_results=3)
        if not results:
            return f"No search results found for '{query}'."

        # Return the snippet from the first result
        first_result_snippet = results[0]['body']
        return first_result_snippet

    except Exception as e:
        return f"Error performing search: {e}"

In [7]:
known_actions ={
    "get_weather": get_weather,
    "ddg_search": ddg_search
}

In [2]:
client =OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

In [27]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({'role': 'system', 'content': self.system})

    def execute(self):
        completion = client.chat.completions.create(
            model="meta-llama/llama-4-maverick-17b-128e-instruct",
            temperature=0,
            messages=self.messages
        )
        return completion.choices[0].message.content

    def run(self, query):
        print(f"🤖 Question: {query}\n")
        self.messages.append({'role': 'user', 'content': query})

        while True:
            result = self.execute()
            self.messages.append({'role': 'assistant', 'content': result})
            
            print(f"🧠 LLM Output:\n---\n{result}\n---\n")

            if "Answer:" in result:
                final_answer = result.split("Answer:")[-1].strip()
                print(f"✅ Final Answer: {final_answer}")
                return
            
            action_match = re.search(r"Action: (\w+): (.*)", result)
            if action_match:
                action_name = action_match.group(1).strip()
                action_input = action_match.group(2).strip()
                
                if action_name in known_actions:
                    action_function = known_actions[action_name]
                    observation = action_function(action_input)
                    observation_message = f"Observation: {observation}"
                    
                    print(f"🛠️  Action: {action_name}('{action_input}')")
                    print(f"👀 {observation_message}\n")
                    
                    self.messages.append({'role': 'user', 'content': observation_message})
                else:
                    print(f"Error: Unknown action '{action_name}'")
                    return
            else:
                print("Error: Could not parse a valid action. Stopping.")
                return

In [28]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

ddg_search:
e.g. ddg_search: Eiffel Tower
Use this to look up factual information about topics, people, and places from an search engine.

get_weather:
e.g. get_weather: Paris
Use this to get the current weather for a specific city.
""".strip()

In [29]:
agent = Agent(prompt)

In [30]:
question = "How is the current weather in Google Headquarters?"
agent.run(question)

🤖 Question: How is the current weather in Google Headquarters?

🧠 LLM Output:
---
Thought: To determine the current weather in Google Headquarters, I need to know the location of Google Headquarters and then check the weather for that location.

Action: ddg_search: Google Headquarters location
PAUSE
---

🛠️  Action: ddg_search('Google Headquarters location')
👀 Observation: September 9, 2025 - The Googleplex is the corporate headquarters complex of Google and its parent company, Alphabet Inc. It is located at 1600 Amphitheatre Parkway in Mountain View, California . The original complex, with 2 million square feet (190,000 square meters) of office space, is the company's second-largest ...

🧠 LLM Output:
---
Thought: Now that I know Google Headquarters is located in Mountain View, California, I can check the current weather for that location.

Action: get_weather: Mountain View
PAUSE
---

🛠️  Action: get_weather('Mountain View')
👀 Observation: The current temperature in Mountain View is 